In [1]:
!pip install mediapipe opencv-python

You should consider upgrading via the 'c:\users\junyup\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

In [7]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        #print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(thickness=2, circle_radius=2),
                                         )
                image_hight, image_width, _ = image.shape
                wrist = np.array([hand.landmark[mp_hands.HandLandmark.WRIST].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.WRIST].y * image_hight])
                thumb = np.array([hand.landmark[mp_hands.HandLandmark.THUMB_TIP].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image_hight])
                index = np.array([hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_hight])
                middle = np.array([hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image_hight])
                ring = np.array([hand.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * image_hight])
                pinky = np.array([hand.landmark[mp_hands.HandLandmark.PINKY_TIP].x * image_width, 
                             hand.landmark[mp_hands.HandLandmark.PINKY_TIP].y * image_hight])
                np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.2f}".format(x)})
                cv2.putText(image, "Thumb "+"{0:0.2f}".format(np.sqrt((thumb[0]-wrist[0])**2+(thumb[1]-wrist[1])**2)), (0,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, " Index "+ "{0:0.2f}".format(np.sqrt((index[0]-wrist[0])**2+(index[1]-wrist[1])**2)), (0,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, "Middle "+ "{0:0.2f}".format(np.sqrt((middle[0]-wrist[0])**2+(middle[1]-wrist[1])**2)), (0,60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, "  Ring "+ "{0:0.2f}".format(np.sqrt((ring[0]-wrist[0])**2+(ring[1]-wrist[1])**2)), (0,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, " Pinky "+ "{0:0.2f}".format(np.sqrt((pinky[0]-wrist[0])**2+(pinky[1]-wrist[1])**2)), (0,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                hand_size = np.array([(hand.landmark[mp_hands.HandLandmark.THUMB_CMC].x-hand.landmark[mp_hands.HandLandmark.PINKY_MCP].x) * image_width, 
                             (hand.landmark[mp_hands.HandLandmark.THUMB_CMC].y-hand.landmark[mp_hands.HandLandmark.PINKY_MCP].y) * image_hight])
                cv2.rectangle(image, (int(thumb[0]), int(middle[1])), (int(pinky[0]), int(wrist[1])), (255, 255, 255), 1)

                #cv2.putText(image, " Hand Size "+str(hand_size), (0,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('Hand Tracking (Press `q` to Quit)', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<img src=https://i.imgur.com/qpRACer.png />

In [ ]:
#mp_hands.HandLandmark.THUMB_TIP